In [ ]:
# !pip install fbprophet

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import dateutil.relativedelta as rd

In [ ]:
# Display all the columns for the dataframes (not-truncated)
pd.set_option("display.max_columns", None)
# Format floats better
pd.options.display.float_format = '{:,.4f}'.format

In [ ]:
covid_df = pd.read_csv("Resources/covid_data.csv")
covid_df

In [ ]:
covid_df = covid_df.groupby(["date"])[[
    "confirmed"
]].sum()
covid_df.index = pd.to_datetime(covid_df.index)
covid_df

In [ ]:
covid_df.plot()

In [ ]:
data_end_date = covid_df.index.max()
# check last week of data
covid_df[(data_end_date - rd.relativedelta(day=7)):].plot(figsize=(12,4))

In [ ]:
# check last month of data
covid_df[(data_end_date - rd.relativedelta(months=1)):].plot(figsize=(12,4))

In [ ]:
covid_df[:(data_end_date - rd.relativedelta(months=1))].plot(figsize=(12,4))

In [ ]:
covid_df[:(data_end_date - rd.relativedelta(months=1))].diff().plot(figsize=(12,4))

In [ ]:
covid_df = covid_df[:(data_end_date - rd.relativedelta(months=1))]
covid_df = covid_df.diff()
covid_df = covid_df.dropna()
covid_df = covid_df.reset_index()
covid_df = covid_df.rename(columns={"confirmed":"y","date":"ds"})
covid_df

In [ ]:
from fbprophet import Prophet

def create_fbprohpet_predictions(df,periods,freq="D"):
    model = Prophet(interval_width=0.95,daily_seasonality=True)
    model_results = model.fit(df)
    future = model.make_future_dataframe(periods=periods,freq=freq)
    extrapolation = model.predict(future)
    return model, extrapolation

In [ ]:
model, extrapolation = create_fbprohpet_predictions(df,periods=365)
extrapolation

In [ ]:
fig = model.plot(extrapolation, figsize=(16,9))

In [ ]:
fig = model.plot_components(extrapolation,figsize=(16,16))

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(
    model,
    initial="365 days",
    period="180 days",
    horizon="365 days"
)
df_cv

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric="rmse")